In [256]:
import numpy as np
import yaml
import pandas as pd    

from dataclasses import dataclass
from typing import List, Optional
import itertools
from docopt import docopt
from prettytable import PrettyTable 
from rich.prompt import Prompt

In [4]:
import logging
logging.basicConfig()
LOGGER = logging.getLogger(__name__)
LOGGER.setLevel(logging.INFO)


In [12]:
def calculate_1rm(reps, weight, rpe):
    """
    Use Brzycki's formula to calculate the theortical 1RM
    """
    one_rm_pct = DF_RPE[f'{reps}'].to_dict().get(rpe)
    one_rm = int(np.round(np.floor(weight / one_rm_pct), 0))
    return one_rm

def round_weights(weight):
    """
    Round the weights to the nearest 2.5 kg (or kg if microplates available)
    """
    if MICROPLATES:
        pl_weight = int(np.floor(weight))
    else: 
        pl_weight = round(np.ceil(weight / 2.5)) * 2.5

    return pl_weight
            

In [253]:

@dataclass
class OneRepMax:
    """A dataclass to store the one rep max for a lift"""

    lift: str
    weight: float
    
    def __post_init__(self):
        """Clean the names / related compound names"""
        exercise_name = self.lift.replace("-", " ").title()
        self.lift = exercise_name
        
    @property
    def output_table(self) -> str:
        """Return a table of the one rep max"""
        output = [self.lift, self.weight]
        return output


@dataclass
class LiftWeight:
    """A dataclass to store the weight for a lift"""
    weight: float
    reps: int
    # pct_one_rm: float

    
@dataclass
class Exercise: 
    """A generic class for an exercise"""
    backoff_sets: List[LiftWeight]
    name: str
    predicted_one_rm: float
    related_compound: str = None
    top_sets: Optional[List[LiftWeight]] = None
    
    def __post_init__(self): 
        """Clean the names / related compound names"""
        exercise_name = self.name.replace('-', ' ').title()
        self.name = exercise_name
        
        if self.related_compound: 
            related_compound_name = self.related_compound.replace('-', ' ').title()
            self.related_compound= related_compound_name
    
    @property 
    def weekly_sets(self) -> List[LiftWeight]:
        """
        Return the weekly training set for the exercise
        """
        if self.top_sets:
            weekly_sets = list(itertools.zip_longest(self.top_sets, self.backoff_sets))
            return weekly_sets
        else: 
            return self.backoff_sets
    
    @property
    def output_name(self) -> str:
        """
        Return the name of the exercise for the output
        """
        if self.related_compound:
            return f'{self.name} ({self.related_compound})'
        else: 
            return self.name
    
    # @property
    # def output_print(self) -> str:
    #     """
    #     Return the string to output to the user
    #     """
        
    #     outputs = [
    #         f"\n{self.output_name}\n"
    #         f"Training Max: {self.predicted_one_rm} kg"]
    #     if self.top_sets:
    #         for week, (top, backoff) in enumerate(self.weekly_sets, 1):
    #             week_date = WEEK_DATES.get(week)
    #             output = f"Week {week} ({week_date})\n"+\
    #             f"{top.weight} kg x {top.reps}\n"+\
    #             f"{backoff.weight} kg x {backoff.reps}"
    #             outputs.append(output)
    #     else: 
    #         for week, weight in enumerate(self.weekly_sets, 1):
    #             week_date = WEEK_DATES.get(week)
    #             output = f"Week {week} ({week_date})\n"+\
    #             f"{weight.weight} kg x {weight.reps}"
    #             outputs.append(output)
    #     return '\n\n'.join(outputs)
    
    @property 
    def output_table(self) -> List: 
        "Creates table output for prettytable package"
        outputs = [
            f"{self.output_name}"
        ]
        if self.top_sets:
            for _, (top, backoff) in enumerate(self.weekly_sets, 1):
                output = f"{top.weight} kg x {top.reps}\n"+\
                f"{backoff.weight} kg x {backoff.reps}"
                outputs.append(output)
        else: 
            for _, weight in enumerate(self.weekly_sets, 1):
                output = f"{weight.weight} kg x {weight.reps}"
                outputs.append(output)
                
        return outputs
    

In [203]:
def has_weekly_difference(training_range): 
    weekly_difference = np.diff(training_range).tolist()
    print(weekly_difference)
    for weekly_diff in weekly_difference:
        print(weekly_diff)
        if weekly_diff < 2.5: 
            return True
    return False

In [230]:
def calculate_training_range(one_rm, reps, rpe_schema) -> List[float]:
    """
    Calculate the training day progression for a given 1RM
    """
    df_ref = DF_RPE[f"{reps}"].to_dict()
    week_weights = []
    pct_one_rms = []
    
    for rpe in rpe_schema:
        pct_one_rm = df_ref.get(rpe)
        week_weight = pct_one_rm * one_rm
        pct_one_rms.append(pct_one_rm)
        week_weights.append(week_weight)

    training_range = [round_weights(i) for i in week_weights]
        
    return training_range

In [272]:
reps = 8
df_ref = DF_RPE[f"{reps}"].to_dict()

lowest_rpe = 6
highest_rpe = 10

one_rm = 110

print(one_rm * df_ref.get(lowest_rpe))
print(one_rm * df_ref.get(highest_rpe))

74.80000000000001
86.46000000000001


In [274]:
np.linspace(70, 80, 5)

array([75.  , 76.25, 77.5 , 78.75, 80.  ])

In [235]:
def create_training_range(one_rm, reps, rpe_schema) -> List[LiftWeight]: 
    """Create a list of LiftWeight objects for the training range"""
    training_range = calculate_training_range(one_rm, reps, rpe_schema)
    # iterate through training range to make sure there is enough spacing between the numbers
    if has_weekly_difference(training_range):
        LOGGER.info("Weekly difference is too small, recalculating")
        lowest_rpe = rpe_schema[0] - 0.5
        highest_rpe = rpe_schema[-1] + 0.5
        updated_rpe_schema = [lowest_rpe] + rpe_schema[1:4] + [highest_rpe]
        training_range=calculate_training_range(one_rm, reps, updated_rpe_schema)
            
    weights_lifted = list(zip(training_range, itertools.repeat(reps)))
    weights_lifted = [LiftWeight(*i) for i in weights_lifted]

    return weights_lifted

In [26]:
from datetime import datetime, timedelta

In [137]:
def create_weekly_dates(start_date): 
    """
    Create the weekly dates for the program
    """
    starting_monday = datetime.date(
        start_date + timedelta(days=7 - start_date.weekday())
    )
    
    week_mondays = []
    for week in range(1, 6):
        week_monday = starting_monday + timedelta(days=7*week)
        week_mondays.append(week_monday)

    # Create the week dates as str 
    week_dates = [i.strftime('%d/%m/%y') for i in week_mondays]                
    return week_dates

In [138]:
create_weekly_dates(datetime.now())

['05/12/22', '12/12/22', '19/12/22', '26/12/22', '02/01/23']

In [30]:
WeeklyExercise(
    week_number=1,
    date=datetime(2021, 1, 1)
)._get_monday

<bound method WeeklyExercise.<lambda> of WeeklyExercise(week_number=1, date=datetime.datetime(2021, 1, 1, 0, 0))>

In [254]:

if __name__ == "__main__":
    # arguments = docopt(__doc__, version="simple-strength-program v1.0")

    # # gather inputs
    # MICROPLATES = False

    # if arguments["--user"]:
    #     # ask for user profile
    #     user_profile = Prompt.ask("Enter user profile name", default="default")
    #     user_profile = user_profile.lower()
    #     user_profile = user_profile.replace(" ", "-")
    # else:
    #     user_profile = "default"
    user_profile='default'
    
    start_date=datetime.now()

    LOGGER.info("Reading source data and configs")
    defined_rpes = yaml.load(open("source/rpe.yaml", "r"), Loader=yaml.FullLoader)
    exercises = yaml.load(open("config/exercise.json", "r"), Loader=yaml.FullLoader)
    accessory_rpe_schema = defined_rpes.get("accessory").get("backoff")

    LOGGER.info("Reading user inputs")
    user_lifts = yaml.load(
        open(f"profiles/{user_profile}/lifts.yaml", "r"), Loader=yaml.FullLoader
    )
    try:
        user_gym = yaml.load(
            open(f"profiles/{user_profile}/gym.yaml", "r"), Loader=yaml.FullLoader
        )
    except FileNotFoundError:
        user_gym = False


    # Global variables
    DF_RPE = pd.read_csv("source/rpe-calculator.csv").set_index("RPE")
    MICROPLATES = user_gym.get("microplates") if user_gym else False
    WEEK_DATES = create_weekly_dates(start_date)
    
    # create weekly dates 
    LOGGER.info("Creating weekly dates")


    max_lifts = []
    full_program = []
    low_accessory_program = []
    high_accessory_program = []

    for compound_lift, stats in user_lifts.items():
        # clear previous stats
        LOGGER.info("Creating program for %s", compound_lift)
        top_training_range = []
        backoff_training_range = []

        lift_one_rep_max = calculate_1rm(
            reps=stats.get("reps"), rpe=stats.get("rpe"), weight=stats.get("weight")
        )

        max_strength = OneRepMax(lift=compound_lift, weight=lift_one_rep_max)

        goal = stats.get("program-goal")
        program_type = goal.split("-")[0]
        rpes = defined_rpes.get(program_type)
        exercise_volume = exercises.get("program-goals").get(f"{goal}")

        if program_type == "strength":
            top_training_range = create_training_range(
                    one_rm=lift_one_rep_max,
                    reps=exercise_volume.get("top-reps"),
                    rpe_schema=rpes.get("top"),
                )
            

        backoff_training_range = create_training_range(
            one_rm=lift_one_rep_max,
            reps=exercise_volume.get("backoff-reps"),
            rpe_schema=rpes.get("backoff"),
        )

        compound = Exercise(
            backoff_sets=backoff_training_range,
            name=compound_lift,
            predicted_one_rm=lift_one_rep_max,
            top_sets=top_training_range,
        )

        full_program.append(compound)

        accessories = exercises.get("accessory-lifts").get(f"{compound_lift}")
        for accessory, stats in accessories.items():
            accessory_one_rm = round_weights(
                lift_one_rep_max * stats.get("max-onerm-pct")
            )
            min_reps = stats.get("min-reps")
            max_reps = stats.get("max-reps")

            low_volume = calculate_training_range(
                one_rm=accessory_one_rm, reps=min_reps, rpe_schema=accessory_rpe_schema
            )

            high_volume = calculate_training_range(
                one_rm=lift_one_rep_max, reps=max_reps, rpe_schema=accessory_rpe_schema
            )

            low_volume_exercise = Exercise(
                backoff_sets=low_volume,
                name=accessory,
                predicted_one_rm=accessory_one_rm,
                related_compound=compound_lift,
            )

            high_volume_exercise = Exercise(
                backoff_sets=high_volume,
                name=accessory,
                predicted_one_rm=lift_one_rep_max,
                related_compound=compound_lift,
            )

            low_accessory_program.append(low_volume_exercise)
            high_accessory_program.append(high_volume_exercise)

        max_lifts.append(max_strength)
    # for exercise in full_program: 
        # print(exercise.output_table
        
        # for acessory in accessory_program: 
            # print(acessory.output_print)

INFO:__main__:Reading source data and configs
INFO:__main__:Reading user inputs
INFO:__main__:Creating weekly dates
INFO:__main__:Creating program for squat
INFO:__main__:Weekly difference is too small, recalculating
INFO:__main__:Creating program for bench
INFO:__main__:Weekly difference is too small, recalculating
INFO:__main__:Creating program for deadlift


[5.0, 2.5, 2.5, 2.5]
5.0
2.5
2.5
2.5
[2.5, 5.0, 2.5, 0.0]
2.5
5.0
2.5
0.0
[5.0, 2.5, 0.0, 2.5]
5.0
2.5
0.0
[10.0, 2.5, 2.5, 2.5]
10.0
2.5
2.5
2.5


In [265]:
x = PrettyTable()
x.field_names = ['Lift', 'Training Max']
for lift in max_lifts:
    max_tm_row = lift.output_table
    x.add_row(max_tm_row)
        

In [266]:
x

Lift,Training Max
Squat,149
Bench,113
Deadlift,209


In [267]:
x = PrettyTable()
WEEK_DATES = create_weekly_dates(start_date)
WEEK_DATES.insert(0, 'Week Starting')
x.add_column("", WEEK_DATES)

In [268]:
x

""
Week Starting
05/12/22
12/12/22
19/12/22
26/12/22
02/01/23


In [269]:
full_program

[Exercise(backoff_sets=[LiftWeight(weight=112.5, reps=4), LiftWeight(weight=117.5, reps=4), LiftWeight(weight=122.5, reps=4), LiftWeight(weight=125.0, reps=4), LiftWeight(weight=127.5, reps=4)], name='Squat', predicted_one_rm=149, related_compound=None, top_sets=[LiftWeight(weight=127.5, reps=2), LiftWeight(weight=132.5, reps=2), LiftWeight(weight=135.0, reps=2), LiftWeight(weight=137.5, reps=2), LiftWeight(weight=140.0, reps=2)]),
 Exercise(backoff_sets=[LiftWeight(weight=77.5, reps=8), LiftWeight(weight=82.5, reps=8), LiftWeight(weight=85.0, reps=8), LiftWeight(weight=85.0, reps=8), LiftWeight(weight=87.5, reps=8)], name='Bench', predicted_one_rm=113, related_compound=None, top_sets=[]),
 Exercise(backoff_sets=[LiftWeight(weight=142.5, reps=8), LiftWeight(weight=152.5, reps=8), LiftWeight(weight=155.0, reps=8), LiftWeight(weight=157.5, reps=8), LiftWeight(weight=160.0, reps=8)], name='Deadlift', predicted_one_rm=209, related_compound=None, top_sets=[])]

In [270]:
for day_no, exercise in enumerate(full_program, 1): 
    
    exercise_col = exercise.output_table
    x.add_column(f"Day {day_no}", exercise_col)

In [271]:
x

,Day 1,Day 2,Day 3
Week Starting,Squat,Bench,Deadlift
05/12/22,127.5 kg x 2112.5 kg x 4,77.5 kg x 8,142.5 kg x 8
12/12/22,132.5 kg x 2117.5 kg x 4,82.5 kg x 8,152.5 kg x 8
19/12/22,135.0 kg x 2122.5 kg x 4,85.0 kg x 8,155.0 kg x 8
26/12/22,137.5 kg x 2125.0 kg x 4,85.0 kg x 8,157.5 kg x 8
02/01/23,140.0 kg x 2127.5 kg x 4,87.5 kg x 8,160.0 kg x 8


In [191]:
for day_no, accessory in enumerate(low_accessory_program,1): 
        accessory_col = accessory.output_table
        # a_table.add_column(f"Option {day_no}", accessory.output_table)

Squat
Squat
Squat
Bench
Bench
Bench
Deadlift


In [189]:
a_table

Day 1,Day 2,Day 3,Day 4,Day 5,Day 6,Day 7
Romanian Deadlifts (Squat),Safety Squat Bar (Squat),Pendulum Squat (Squat),Paused Bench (Bench),3 Count Paused Bench (Bench),Incline Bench Press (Bench),Conventional Deadlifts (Deadlift)
85.0 kg x 6,92.5 kg x 3,52.5 kg x 8,65.0 kg x 3,62.5 kg x 3,75.0 kg x 4,132.5 kg x 4
87.5 kg x 6,95.0 kg x 3,55.0 kg x 8,67.5 kg x 3,65.0 kg x 3,77.5 kg x 4,137.5 kg x 4
90.0 kg x 6,97.5 kg x 3,57.5 kg x 8,70.0 kg x 3,65.0 kg x 3,77.5 kg x 4,142.5 kg x 4
90.0 kg x 6,100.0 kg x 3,57.5 kg x 8,72.5 kg x 3,67.5 kg x 3,80.0 kg x 4,142.5 kg x 4
92.5 kg x 6,102.5 kg x 3,57.5 kg x 8,72.5 kg x 3,67.5 kg x 3,80.0 kg x 4,145.0 kg x 4


In [84]:
training_range = [80, 85, 87.5, 92.5, 95]
upper_limit = training_range[-1]
lower_limit = training_range[0]

In [98]:
for exercise in full_program: 
    print(exercise.output_print)
    # for set in exercise.top_sets: 
    #     print(set.weight, set.reps, set.rpe)
        
    # for set in exercise.backoff_sets: 
    #     print(set.weight, set.reps, set.rpe)

Squat
Training max: 149 kg
W1: 120.0 kg x 2
W1: 110.0 kg x 4
W2: 125.0 kg x 2
W2: 115.0 kg x 4
W3: 130.0 kg x 2
W3: 117.5 kg x 4
W4: 135.0 kg x 2
W4: 122.5 kg x 4
W5: 140.0 kg x 2
W5: 125.0 kg x 4
Bench
Training max: 113 kg
W1: 70.0 kg x 8
W2: 75.0 kg x 8
W3: 80.0 kg x 8
W4: 85.0 kg x 8
W5: 90.0 kg x 8
Deadlift
Training max: 209 kg
W1: 140.0 kg x 8
W2: 145.0 kg x 8
W3: 150.0 kg x 8
W4: 155.0 kg x 8
W5: 160.0 kg x 8
